In [1]:
from ros_edf.ros_interface import EdfRosInterface
from edf.data import PointCloud, SE3, TargetPoseDemo, DemoSequence, DemoSeqDataset, save_demos
from edf.pc_utils import optimize_pcd_collision, draw_geometry
from edf.preprocess import Rescale, NormalizeColor, Downsample, ApplySE3
from edf.agent import PickAgent, PlaceAgent

import torch
from torch.utils.data import DataLoader
from torchvision.transforms import Compose

torch.set_printoptions(precision= 3, sci_mode=False, linewidth=120)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Initialize Robot Interface

In [2]:
env_interface = EdfRosInterface(reference_frame = "scene")
env_interface.reset()
env_interface.moveit_interface.arm_group.set_planning_time(seconds=5)

[ WARN] [1676268174.804222163]: Link axle_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1676268174.804242319]: Link front_rocker_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1676268174.804254075]: Link rear_rocker_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1676268174.804263875]: Link front_cover_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1676268174.804269385]: Link front_lights_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying colli

[INFO] [1676268176.056413]: Commencing scene point cloud update...
[INFO] [1676268176.407205]: Processing received scene point cloud...
[INFO] [1676268176.412616]: Scene pointcloud update success!
[INFO] [1676268176.413166]: Commencing end-effector point cloud update...
[INFO] [1676268176.463692]: Processing received end-effector point cloud...
[INFO] [1676268176.464459]: End-effector pointcloud update success!


In [3]:
plan_targets = [(SE3([0., 0., 1., 0., 0., 0., 0.6]), "default", {}),
                (SE3([0., 0., 1., 0., 0., 0., 0.4]), "default", {}),
                (SE3([0., 0., 1., 0., 0., 0., 0.6]), "default", {}),
                (SE3([[0., 0., 1., 0., 0., 0., 0.4],[0., 0., 1., 0., 0., 0., 0.6]]), "cartesian", {'cartesian_step': 0.01, 'cspace_step_thr': 10., 'avoid_collision': False, 'success_fraction': 0.95}),
                (SE3([[0., 0., 1., 0., 0., 0., 0.4],[0., 0., 1., 0., 0., 0., 0.6]]), "cartesian", {'cartesian_step': 0.01, 'cspace_step_thr': 10., 'avoid_collision': False, 'success_fraction': 0.95}),
                (SE3([0., 0., 1., 0., 0., 0., 0.5]), "default", {}),
               ]


plan_results, plans = env_interface.move_plans(targets=plan_targets)

[INFO] [1676268180.963775]: EDF Moveit Interface: Found a motion plan with length: 10
[INFO] [1676268181.266529]: EDF Moveit Interface: Found a motion plan with length: 6
[INFO] [1676268181.363917]: EDF Moveit Interface: Found a motion plan with length: 6
[INFO] [1676268181.398805]: EDF Moveit Interface: Found a cartesian plan with length: 43
[INFO] [1676268181.448874]: EDF Moveit Interface: Found a cartesian plan with length: 42
[INFO] [1676268181.567001]: EDF Moveit Interface: Found a motion plan with length: 4


In [4]:
if plan_results[-1] is True:
    env_interface.execute_plans(plans)